In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# category variable
category = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']

# continuous variable
continuous = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 
              'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']

In [2]:
#features = ['cat1', 'cont0', 'cat2', 'cont11', 'cont13', 'cat8', 'cont8', 'cont1', 'cat9',
#            'cont9', 'cont5', 'cat3', 'cat0', 'cat6', 'cont3', 'cat5', 'cont4', 'cont2', 'cont12']

train_dataset = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
dataset = pd.concat([train_dataset, test_data])

# あきらかな外れ値は削除
# [166042]はあきらかな外れ値
outlier=[166042]
for x in outlier:
    dataset = dataset.loc[dataset['id'] != x, :]

# idとtargetは避難させておく
id = dataset['id']
target = dataset['target']
# 避難させたので遠慮なく削除
dataset = dataset.drop(columns=['id', 'target'])
# 相関が高いものだけを使用
# この手法は有効だったと思うが、Catboostでのスコアは一定以上伸びなかった。

# 重要度を確認するためにはLabelEncodingが有効
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
encoder = LabelEncoder()
scaler = RobustScaler()
#scaler = StandardScaler()

for x in dataset.columns:
    # notebookみると、みんなLabelEncoder使っているなぁ
    if dataset[x].dtype == object:
        #dataset[x] = encoder.fit_transform(dataset[x])
        dataset = pd.get_dummies(dataset, columns=[x], drop_first=True)
        
#dataset[continuous] = scaler.fit_transform(dataset.loc[:,continuous].values)

# データセットにidとtargetを元に戻して
dataset = pd.concat([id,dataset,target], axis=1)
# targetのあるなしでtrainとtestを分割
train = dataset.loc[dataset['target'].notnull(), :]
test  = dataset.loc[dataset['target'].isnull(), :]

X = train.drop(columns=['id', 'target'])
y = train['target']
X_prediction = test.drop(columns=['id', 'target'])
prediction_id = test.loc[:,'id']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [3]:
## Lightgbm Optuna

#!pip install optuna 
import optuna.integration.lightgbm as lgb

trains = lgb.Dataset(X_train, y_train)
tests = lgb.Dataset(X_test, y_test)

params = {'objective': 'mean_squared_error',
         'metric': 'rmse'}

model = lgb.train(params, trains, valid_sets=tests, early_stopping_rounds=10)
best_params = model.params


import lightgbm as lgb

regressor = lgb.train(best_params, trains, valid_sets=tests)
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, y_pred))

[I 2021-02-05 22:50:58,696] A new study created in memory with name: no-name-c38c9714-0ce2-41ed-8978-51eecaaadeb7
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022568 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3662
[LightGBM] [Info] Number of data points in the train set: 239999, number of used features: 60
[LightGBM] [Info] Start training from score 7.457212
[1]	valid_0's rmse: 0.888383
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 0.885634
[3]	valid_0's rmse: 0.883426
[4]	valid_0's rmse: 0.881277
[5]	valid_0's rmse: 0.879457
[6]	valid_0's rmse: 0.877413
[7]	valid_0's rmse: 0.875946
[8]	valid_0's rmse: 0.874716
[9]	valid_0's rmse: 0.873567
[10]	valid_0's rmse: 0.87252
[11]	valid_0's rmse: 0.87115
[12]	valid_0's rmse: 0.869951
[13]	valid_0's rmse: 0.868912
[14]	valid_0's rmse: 0.867893
[15]	valid_0's rmse: 0.867223
[16]	valid_0's rmse: 0.866448
[17]	valid_0's rmse: 0.865629
[18]	valid_0's rmse: 0.864958


feature_fraction, val_score: 0.847706:  14%|#4        | 1/7 [00:05<00:31,  5.20s/it]

[213]	valid_0's rmse: 0.847709
[214]	valid_0's rmse: 0.847708
[215]	valid_0's rmse: 0.847707
[216]	valid_0's rmse: 0.847706
[217]	valid_0's rmse: 0.847718
[218]	valid_0's rmse: 0.84771
[219]	valid_0's rmse: 0.847723
[220]	valid_0's rmse: 0.847737
[221]	valid_0's rmse: 0.847732
[222]	valid_0's rmse: 0.847748
[223]	valid_0's rmse: 0.847759
[224]	valid_0's rmse: 0.847768
[225]	valid_0's rmse: 0.847771
[226]	valid_0's rmse: 0.847769
Early stopping, best iteration is:
[216]	valid_0's rmse: 0.847706
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.068561 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3662
[LightGBM] [Info] Number of data points in the train set: 239999, number of used features: 60
[LightGBM] [Info] Start training from score 7.457212
[1]	valid_0's rmse: 0.887489
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 0.884451
[3]	valid_0's rmse: 0.881814
[4]	val

feature_fraction, val_score: 0.847706:  29%|##8       | 2/7 [00:09<00:22,  4.55s/it]

...

min_data_in_leaf, val_score: 0.847465: 100%|##########| 5/5 [00:25<00:00,  5.00s/it]

[183]	valid_0's rmse: 0.847919
[184]	valid_0's rmse: 0.847928
[185]	valid_0's rmse: 0.84791
[186]	valid_0's rmse: 0.847892
[187]	valid_0's rmse: 0.8479
[188]	valid_0's rmse: 0.847902
[189]	valid_0's rmse: 0.847921
[190]	valid_0's rmse: 0.847918
[191]	valid_0's rmse: 0.847944
[192]	valid_0's rmse: 0.847934
[193]	valid_0's rmse: 0.847914
[194]	valid_0's rmse: 0.847924
[195]	valid_0's rmse: 0.847929
[196]	valid_0's rmse: 0.847943
Early stopping, best iteration is:
[186]	valid_0's rmse: 0.847892


[I 2021-02-05 22:56:47,365] Trial 64 finished with value: 0.8478917517714692 and parameters: {'min_child_samples': 5}. Best is trial 63 with value: 0.8475794107209788.
min_data_in_leaf, val_score: 0.847465: 100%|##########| 5/5 [00:25<00:00,  5.06s/it]
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019157 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3662
[LightGBM] [Info] Number of data points in the train set: 239999, number of used features: 60
[LightGBM] [Info] Start training from score 7.457212
[1]	valid_0's rmse: 0.888331
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 0.885546
[3]	valid_0's rmse: 0.883304
[4]	valid_0's rmse: 0.881132
[5]	valid_0's rmse: 0.879271
[6]	valid_0's rmse: 0.877206
[7]	valid_0's rmse: 0.875721
[8]	valid_0's rmse: 0.874468
[9]	valid_0's rmse: 0.873307
[10]	valid_0's rmse: 0.87227
[11]	valid_0's rmse: 0.870894
[12]	valid_0's rmse: 0.869691
[13]	valid_0's rmse: 0.868656
[14]	valid_0's rmse: 0.867618
[15]	valid_0's rmse: 0.866922
[16]	valid_0's rmse: 0.866149
[17]	valid_0's rmse: 0.865324
[18]	valid_0's rmse: 0.8647
[

[256]	valid_0's rmse: 0.847466
[257]	valid_0's rmse: 0.847479
[258]	valid_0's rmse: 0.847489
[259]	valid_0's rmse: 0.847487
[260]	valid_0's rmse: 0.84747
[261]	valid_0's rmse: 0.847473
[262]	valid_0's rmse: 0.847469
[263]	valid_0's rmse: 0.8475
[264]	valid_0's rmse: 0.847498
[265]	valid_0's rmse: 0.847511
Early stopping, best iteration is:
[255]	valid_0's rmse: 0.847465
0.7181975070303817


In [4]:
from sklearn.model_selection import KFold
import lightgbm as lgb

kf = KFold(n_splits=5,random_state=48,shuffle=True)
rmse=[]  # list contains rmse for each fold
n=0
preds=0

# kf.splitはindexを返すことに注意。データ自体じゃないよ!
for train, test in kf.split(X, y):
    model = lgb.LGBMRegressor(**best_params)
    x_tr = X.iloc[train, :].values
    x_te = X.iloc[test,  :].values
    y_tr = y.iloc[train].values
    y_te = y.iloc[test].values
    
    model.fit(x_tr,y_tr,eval_set=[(x_te,y_te)],early_stopping_rounds=100,verbose=False)
    rmse.append(mean_squared_error(y_te, model.predict(x_te), squared=False))
    preds+=model.predict(X_prediction)
    print(n+1,rmse[n])
    n+=1

y_pred = preds/kf.n_splits
import statistics as stat
print(f'RMSE mean = {stat.mean(rmse)}')
print(f'RMSE = {rmse}')

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] lambda_l1 is set=3.091223000030874e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.091223000030874e-06
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.0005486056579747929, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0005486056579747929
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
1 0.8423521171301173
[LightGBM] [Warning] lambda_l1 is set=3.091223000030874e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.091223000030874e-06
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.0005486056579747929, reg_lambda=0.0 will be ignored. Cu

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


2 0.8463045408944343
[LightGBM] [Warning] lambda_l1 is set=3.091223000030874e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.091223000030874e-06
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.0005486056579747929, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0005486056579747929
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


3 0.8466526687737727
[LightGBM] [Warning] lambda_l1 is set=3.091223000030874e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.091223000030874e-06
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.0005486056579747929, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0005486056579747929
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


4 0.8434694053761003
[LightGBM] [Warning] lambda_l1 is set=3.091223000030874e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.091223000030874e-06
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.0005486056579747929, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0005486056579747929
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


5 0.8432270968880907
RMSE mean = 0.844401165812503
RMSE = [0.8423521171301173, 0.8463045408944343, 0.8466526687737727, 0.8434694053761003, 0.8432270968880907]


In [ ]:
## CatBoost Optuna
from catboost import CatBoostRegressor
from catboost import Pool
from sklearn.metrics import r2_score, mean_squared_error
import optuna

cat_train = Pool(X_train, y_train, cat_features=category)
cat_test = Pool(X_test, y_test, cat_features=category)

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 500),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'l2_leaf_reg': trial.suggest_int('l2_leaf', 1,10),
        'eval_metric': trial.suggest_categorical('eval_metric', ['RMSE']),
        'random_strength': trial.suggest_int('random_strength', 0, 100),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait': trial.suggest_int('od_wait', 10, 50),
        'use_best_model': True,
        'cat_features': category
    }  
    
    model = CatBoostRegressor(**params)
    model.fit(cat_train, eval_set = cat_test)
    y_pred = model.predict(cat_test)
    
    return np.sqrt(mean_squared_error(y_test, y_pred))

study = optuna.create_study()
study.optimize(objective, n_trials=20, n_jobs=-1)
cat_best = study.best_params
print(cat_best)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import r2_score, mean_squared_error

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    
    def fit(self, X, y, X_test, y_test):
        self.models_ = [clone(x) for x in self.models]

        for model in self.models_:
            # CatBoostに関しては、eval_setが必須のためここでエラーが発生する。
            # その場合はexceptでfit処理を行う
            print(f'************ RUNNING : {model} ************')
            try:
                model.fit(X, y)
            except:
                model.fit(X, y, eval_set=(X_test, y_test))
                
            y_pred = model.predict(X_test)
            print(f'{model} RMSE: {mean_squared_error(y_test, y_pred)}')
            
        return self
    
    def predict(self, X):
        predictions = np.column_stack(
            [model.predict(X) for model in self.models_]
            )
        return np.mean(predictions, axis=1)

    
averaged_models = AveragingModels(models = (cat, lasso, lgb_tuned))
averaged_models.fit(X_train, y_train, X_test, y_test)

# 予想してみる
from sklearn.metrics import mean_squared_error
y_pred = averaged_models.predict(X_test)

In [ ]:
import tensorflow as tf
from tensorflow import keras

def rmse(y_true, y_pred):
    return tf.sqrt(tf.losses.mean_squared_error(y_true, y_pred))

### ***********************************************************###
model = tf.keras.Sequential()

#model.add(tf.keras.layers.Dense(1024, activation='relu', input_shape=(X.shape[1],)))
#model.add(tf.keras.layers.LeakyReLU())
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(0.5))

#model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu',input_shape=(X.shape[1],)))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

#model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

#model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(32, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

#model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(32, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

#model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())

model.add(tf.keras.layers.Dense(units=1, activation='linear'))
### ***********************************************************###

optimizer = tf.keras.optimizers.Adam(lr=0.005, decay=5e-4)
model.compile(optimizer = optimizer, loss = 'mae', metrics = ['mse', 'mae'])

#checkpoint_name = 'Model/{epoch:03d}-{val_loss:.5f}.hdf5'
checkpoint_name = 'DNN_BestModel.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callback_list = [checkpoint]

history = model.fit(X_train, y_train, validation_split=0.2, epochs = 500, batch_size = 1024,
                    validation_data=(X_test, y_test), callbacks=callback_list)
y_pred = model.predict(X_test).reshape(-1)
RMSE = rmse(y_test, y_pred)
print(f'RMSE = {RMSE}')

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
ax.set_ylim(0.6, 0.75)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Train', 'Test'], loc='upper right')
plt.savefig(fname='1024 neurons enable LearningRate.png')

In [ ]:
# CatBoosting
'''
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.model_selection import StratifiedKFold, KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

from sklearn.model_selection import GridSearchCV
param_cat = {'depth':[6,8,10],
            'learning_rate':[0.005, 0.001],
            'l2_leaf_reg':[1,4,9],
            'iterations':[100],
            'cat_features':[feature_cat],
            'eval_metric':['RMSE']
            }

grid_result = GridSearchCV(estimator=CatBoostRegressor(),param_grid=param_cat, cv=kfold, scoring='neg_mean_squared_error', n_jobs = -1, verbose=2)
grid_result.fit(X_train, y_train)
grid_param = grid_result.best_params_
print(grid_param)


cat = CatBoostRegressor(task_type='GPU', iterations=8000, use_best_model=True, depth=10, eval_metric='RMSE', l2_leaf_reg=1, learning_rate=0.001, early_stopping_rounds=10)
cat.fit(X_train, y_train, cat_features=feature_cat, eval_set = (X_test, y_test))
y_pred = cat.predict(X_test)
print(f'RMSE: {mean_squared_error(y_test, y_pred)}')


'''

In [ ]:
#y_pred=regressor.predict(X_prediction)
output = pd.DataFrame({'id': prediction_id, 'target': y_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")